In [1]:
import h3
import folium
import numpy as np

from pathfinder.algo import (
    OutwardSpiralPathFinder, 
    BayesianHexSearch
)
from utils.hex import (
    hex_to_binary, 
    octal_list_to_binary, 
    binary_to_octal_list, 
    binary_to_hex, 
    hex_to_array_index,
    array_index_to_hex,
    distance_between_2_hexas,
)
from utils.viz import (
    gradient_color,
    add_hex_to_map
)
from experiment.test_framework import TestFramework

In [2]:
CENTRE = (1.340640225800291, 103.962424829341)
RES = 15
STEPS = 500
f = 0.99

test_spiral = TestFramework(name="Spiral", waypoint=CENTRE, res=RES)
test_spiral.register_pathfinder(OutwardSpiralPathFinder)
spiral_output = test_spiral.run(STEPS)
test_spiral.evaluate()

test_bayes = TestFramework(name="Bayes", waypoint=CENTRE, res=RES, f=f)
test_bayes.register_pathfinder(BayesianHexSearch)
bayes_output = test_bayes.run(STEPS, update_map=True)
test_bayes.evaluate()


Spiral's Path Coverage: 100.0%
Bayes's Path Coverage: 98.0%


In [3]:
m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)
add_hex_to_map(spiral_output, m)
display(m)

In [4]:
m = folium.Map(location=[CENTRE[0],CENTRE[1]], zoom_start=23, max_zoom=25)
add_hex_to_map(bayes_output, m)
display(m)